In [1]:
import numpy as np
import os

In [2]:
eval_size = [256, 256, 16]

LABELS_REMAP = np.array([
    0,  # Free
    1,  # Building
    2,  # Barrier
    3,  # Other
    4,  # Pedestrian
    5,  # Pole or Traffic Light/Sign
    6,  # Roadline -> Road
    6,  # Road
    8,  # Sidewalk
    9,  # Vegetation
    10,  # Vehicles
    2,  # Wall -> Barrier
    5,  # Traffic Sign -> Pole
    3,  # Sky -> Other
    7,  # Ground
    3,  # Bridge -> Other
    3,  # Railtrack -> Other
    2,  # GuardRail -> Barrier
    5,  # Traffic Light -> Pole
    3,  # Static -> Other
    3,  # Dynamic -> Other
    3,  # Water -> Other
    7,  # Terrain -> Ground
])

In [3]:
def form_bev(labels, counts):
    bev_map = np.zeros((eval_size[0], eval_size[1]))
    for x in range(eval_size[0]):
        for y in range(eval_size[1]):
            label_xy = None
            for z in range(eval_size[2]-1, -1, -1):
                count_z = counts[x, y, z]
                if count_z != 0 and labels[x, y, z] != 0:
                    intersected = True
                    label_xy = labels[x, y, z]
                    break
            if label_xy:
                remapped_xy = LABELS_REMAP[label_xy]
                bev_map[x, y] = remapped_xy
    return bev_map.astype(np.uint8)

In [4]:
file_path = "/home/tigeriv/Data/Carla/Cartesian"
for split in ["Train", "Val", "Test"]:
    split_path = os.path.join(file_path, split)
    seqs = os.listdir(split_path)
    for seq in seqs:
        seq_path = os.path.join(split_path, seq)
        bev_path = os.path.join(seq_path, "cartesian", "bev_gt")
        if not os.path.exists(bev_path):
            os.mkdir(bev_path)
        eval_path = os.path.join(seq_path, "cartesian", "evaluation_fine")
        files = os.listdir(eval_path)
        num_files = (len(files) - 1) // 2
        for i in range(num_files):
            counts_file = os.path.join(eval_path, str(i).zfill(6) + ".bin")
            labels_file = os.path.join(eval_path, str(i).zfill(6) + ".label")
            
            labels = np.fromfile(labels_file,dtype=np.uint8).reshape(eval_size)
            counts = np.fromfile(counts_file,dtype=np.int8).reshape(eval_size)
    
            bev_map = form_bev(labels, counts)
            bev_file = os.path.join(bev_path, str(i).zfill(6) + ".bin")
            bev_map.tofile(bev_file)